<a href="https://colab.research.google.com/github/kuruvagantelahari/Evaluating-Classical-Machine-Learning-Models-for-Topic-Classification-in-High-Dimensional-Text/blob/main/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix
)

from joblib import dump
# ======================================================
# LOAD HF DATASET (FIX MISSING TEXT)
# ======================================================
def load_data():
    print("Loading TopicNet/20-Newsgroups dataset...")
    ds = load_dataset("TopicNet/20-Newsgroups")

    text_col = "raw_text"
    label_col = "target"

    X_train_raw = ds["train"][text_col]
    y_train_raw = ds["train"][label_col]

    X_test_raw = ds["test"][text_col]
    y_test_raw = ds["test"][label_col]

    # ---------- FIX: Remove None / invalid ----------
    clean_train = [(x, y) for x, y in zip(X_train_raw, y_train_raw)
                   if x is not None and isinstance(x, str)]
    clean_test = [(x, y) for x, y in zip(X_test_raw, y_test_raw)
                  if x is not None and isinstance(x, str)]

    X_train, y_train = zip(*clean_train)
    X_test, y_test = zip(*clean_test)

    print(f"Cleaned Train samples: {len(X_train)}, Cleaned Test samples: {len(X_test)}")

    return list(X_train), list(X_test), list(y_train), list(y_test)

# ======================================================
# TF-IDF VECTORISER
# ======================================================
def build_vectorizer():
    return TfidfVectorizer(
        min_df=3,
        max_df=0.9,
        ngram_range=(1, 2),
        sublinear_tf=True,
        stop_words="english"
    )

# ======================================================
# MODEL EVALUATION
# ======================================================
def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    print(f"\nTraining {name}...")

    start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start

    preds = model.predict(X_test)

    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, average="macro")
    prec = precision_score(y_test, preds, average="macro")
    rec = recall_score(y_test, preds, average="macro")

    print(f"{name} | Accuracy: {acc:.4f} | F1: {f1:.4f} | Time: {train_time:.2f}s")

    # ---- Confusion Matrix ----
    cm = confusion_matrix(y_test, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, cmap="Blues", annot=False)
    plt.title(f"Confusion Matrix - {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"confusion_matrix_{name}.png", dpi=200)
    plt.show()
    plt.close()

    return acc, f1, prec, rec, train_time

# ======================================================
# MAIN EXPERIMENT PIPELINE
# ======================================================
def run_experiment():

    # Load dataset
    X_train_raw, X_test_raw, y_train, y_test = load_data()

    # TF-IDF
    print("\nVectorizing using TF-IDF...")
    tfidf = build_vectorizer()
    X_train = tfidf.fit_transform(X_train_raw)
    X_test = tfidf.transform(X_test_raw)
    print("TF-IDF shape:", X_train.shape)

    # Models
    svc = LinearSVC(max_iter=5000)
    lr = LogisticRegression(max_iter=2000, solver="saga", n_jobs=-1)
    nb = MultinomialNB()

    results = []

    # ---------- Evaluate Full TF-IDF ----------
    results.append(("LinearSVC",) +
                   evaluate_model("LinearSVC", svc, X_train, y_train, X_test, y_test))

    results.append(("LogisticRegression",) +
                   evaluate_model("LogisticRegression", lr, X_train, y_train, X_test, y_test))

    results.append(("MultinomialNB",) +
                   evaluate_model("MultinomialNB", nb, X_train, y_train, X_test, y_test))

    # ======================================================
    # APPLY TRUNCATED SVD
    # ======================================================
    print("\nApplying TruncatedSVD (300 components)...")
    svd = TruncatedSVD(n_components=300, random_state=42)
    X_train_svd = svd.fit_transform(X_train)
    X_test_svd = svd.transform(X_test)
    print("SVD shape:", X_train_svd.shape)

    # ---------- Evaluate SVD Models ----------
    results.append(("LinearSVC SVD-300",) +
                   evaluate_model("LinearSVC SVD-300", svc, X_train_svd, y_train, X_test_svd, y_test))

    results.append(("LogReg SVD-300",) +
                   evaluate_model("LogReg SVD-300", lr, X_train_svd, y_train, X_test_svd, y_test))

    # Save results to CSV
    df = pd.DataFrame(results, columns=["Model", "Accuracy", "MacroF1", "Precision", "Recall", "TrainTime"])
    df.to_csv("results_summary.csv", index=False)
    print("\nSaved: results_summary.csv")

    # ======================================================
    # SAVE MODELS
    # ======================================================
    dump(svc, "model_LinearSVC.joblib")
    dump(lr, "model_LogReg.joblib")
    dump(nb, "model_NaiveBayes.joblib")
    dump(tfidf, "tfidf_vectorizer.joblib")
    dump(svd, "svd_transformer.joblib")
    print("Saved model artifacts.")

    # ======================================================
    # COMPARISON PLOTS
    # ======================================================
    metrics = ["Accuracy", "MacroF1", "TrainTime"]
    filenames = ["accuracy_comparison.png", "f1_comparison.png", "train_time_comparison.png"]

    for metric, file in zip(metrics, filenames):
        plt.figure(figsize=(10, 5))
        plt.bar(df["Model"], df[metric])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel(metric)
        plt.title(f"{metric} Comparison Across Models")
        plt.tight_layout()
        plt.savefig(file, dpi=200)
        plt.show()
        plt.close()

    # ======================================================
    # PCA SCATTERPLOT
    # ======================================================
    print("Generating PCA scatterplot...")

    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_train_svd[:500])

    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1],
                          c=y_train[:500], cmap="tab20", s=20)
    plt.colorbar(scatter)
    plt.title("2D PCA Projection of SVD-300 Features (Sample 500)")
    plt.tight_layout()
    plt.savefig("pca_scatterplot.png", dpi=200)
    plt.show()
    plt.close()

    print("\nPipeline completed successfully!")

# ======================================================
# MAIN EXECUTION
# ======================================================
if __name__ == "__main__":
    run_experiment()

SyntaxError: invalid non-printable character U+00A0 (ipython-input-2591534739.py, line 14)

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## **Introduction to Colab and Python**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Welcome to this Colab where you will get a quick introduction to the Python programming language and the environment used for the course's exercises: Colab.

Colab is a Python development environment that runs in the browser using Google Cloud.

For example, to print "Hello World", just hover the mouse over [ ] and press the play button to the upper left. Or press shift-enter to execute.

In [ ]:
print("Hello World")

## Functions, Conditionals, and Iteration
Let's create a Python function, and call it from a loop.

In [ ]:
def HelloWorldXY(x, y):
  if (x < 10):
    print("Hello World, x was < 10")
  elif (x < 20):
    print("Hello World, x was >= 10 but < 20")
  else:
    print("Hello World, x was >= 20")
  return x + y

for i in range(8, 25, 5):  # i=8, 13, 18, 23 (start, stop, step)
  print("--- Now running with i: {}".format(i))
  r = HelloWorldXY(i,i)
  print("Result from HelloWorld: {}".format(r))

In [ ]:
print(HelloWorldXY(1,2))

Easy, right?

If you want a loop starting at 0 to 2 (exclusive) you could do any of the following

In [ ]:
print("Iterate over the items. `range(2)` is like a list [0,1].")
for i in range(2):
  print(i)

print("Iterate over an actual list.")
for i in [0,1]:
  print(i)

print("While works")
i = 0
while i < 2:
  print(i)
  i += 1

In [ ]:
print("Python supports standard key words like continue and break")
while True:
  print("Entered while")
  break

## Numpy and lists
Python has lists built into the language.
However, we will use a library called numpy for this.
Numpy gives you lots of support functions that are useful when doing Machine Learning.

Here, you will also see an import statement. This statement makes the entire numpy package available and we can access those symbols using the abbreviated 'np' syntax.

In [ ]:
import numpy as np  # Make numpy available using np.

# Create a numpy array, and append an element
a = np.array(["Hello", "World"])
a = np.append(a, "!")
print("Current array: {}".format(a))
print("Printing each element")
for i in a:
  print(i)

print("\nPrinting each element and their index")
for i,e in enumerate(a):
  print("Index: {}, was: {}".format(i, e))

In [ ]:
print("\nShowing some basic math on arrays")
b = np.array([0,1,4,3,2])
print("Max: {}".format(np.max(b)))
print("Average: {}".format(np.average(b)))
print("Max index: {}".format(np.argmax(b)))

In [ ]:
print("\nYou can print the type of anything")
print("Type of b: {}, type of b[0]: {}".format(type(b), type(b[0])))

In [ ]:
print("\nUse numpy to create a [3,3] dimension array with random number")
c = np.random.rand(3, 3)
print(c)

In [ ]:
print("\nYou can print the dimensions of arrays")
print("Shape of a: {}".format(a.shape))
print("Shape of b: {}".format(b.shape))
print("Shape of c: {}".format(c.shape))
print("...Observe, Python uses both [0,1,2] and (0,1,2) to specify lists")

## Colab Specifics

Colab is a virtual machine you can access directly. To run commands at the VM's terminal, prefix the line with an exclamation point (!).


In [ ]:
print("\nDoing $ls on filesystem")
!ls -l
!pwd

In [ ]:
print("Install numpy")  # Just for test, numpy is actually preinstalled in all Colab instances
!pip install numpy

**Exercise**

Create a code cell underneath this text cell and add code to:


*   List the path of the current directory (pwd)
* Go to / (cd) and list the content (ls -l)

In [ ]:
!pwd
!cd /
!ls -l
print("Hello")

All usage of Colab in this course is completely free or charge. Even GPU usage is provided free of charge for some hours of usage every day.

**Using GPUs**
* Many of the exercises in the course executes more quickly by using GPU runtime: Runtime | Change runtime type | Hardware accelerator | GPU

**Some final words on Colab**
*   You execute each cell in order, you can edit & re-execute cells if you want
*   Sometimes, this could have unintended consequences. For example, if you add a dimension to an array and execute the cell multiple times, then the cells after may not work. If you encounter problem reset your environment:
  *   Runtime -> Restart runtime... Resets your Python shell
  *   Runtime -> Restart all runtimes... Will reset the Colab image, and get you back to a 100% clean environment
* You can also clear the output in the Colab by doing: Edit -> Clear all outputs
* Colabs in this course are loaded from GitHub. Save to your Google Drive if you want a copy with your code/output: File -> Save a copy in Drive...

**Learn More**
*   Check out [this](https://www.youtube.com/watch?v=inN8seMm7UI&list=PLQY2H8rRoyvwLbzbnKJ59NkZvQAW9wLbx&index=3) episode of #CodingTensorFlow, and don't forget to subscribe to the YouTube channel ;)
